# Introduction

This notebook helps you get started exploring Transformers using the HuggingFace library. 

Have a look at the great documentation over at https://huggingface.co/transformers. 

# Setup

In [1]:
gradient=False

In [19]:
%matplotlib inline
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from glob import glob
from pathlib import Path
import shutil
import os, sys
import urllib

In [3]:
if gradient:
    !pip install -Uqq fastbook
    import fastbook
    fastbook.setup_book()
    from fastbook import *
    !pip install transformers

In [14]:
if gradient:
    DATA = Path('/storage/huggingface')
    DATA.mkdir(exist_ok=True)
    TRANSFORMERS_CACHE=DATA/'cache'
else:
    DATA=Path('/home/alex/data-tmp/huggingface')
    DATA.mkdir(exist_ok=True)

In [28]:
from fastai.basics import *

In [7]:
# Verify installation
import transformers
print(transformers.pipeline('sentiment-analysis')('we love you'))

[{'label': 'POSITIVE', 'score': 0.9998704791069031}]


# Get a pretrained model and tokenizer

We'll use the [PubMedBERT model](https://huggingface.co/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract) created by Microsoft Research by training a BERT model on 14 million abstracts of PubMed articles. 

Have a look at the blog post [Domain-specific language model pretraining for biomedical natural language processing](https://www.microsoft.com/en-us/research/blog/domain-specific-language-model-pretraining-for-biomedical-natural-language-processing/) and the accompanying paper. 

In [10]:
model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract'
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSeque

In [11]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/225k [00:00<?, ?B/s]

In [12]:
tokenizer("""Our findings indicate that the @GENE$-112G/A polymorphism 
          does not play a substantial role in genetic predisposition to @DISEASE$ in this Japanese population.""")

{'input_ids': [2, 2280, 2606, 3275, 1760, 1680, 35, 2397, 7, 16, 12851, 1013, 18, 42, 6343, 4042, 1888, 3568, 42, 5945, 2467, 1682, 3056, 15831, 1701, 35, 2174, 7, 1682, 1805, 7910, 2806, 17, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

# Genetic Association Database (GAD)

Let's try to fine-tune the model to perform a new task. We'll use GAD, which contains gene-disease relations, based on the data set prepeared by BioBERT: https://github.com/dmis-lab/biobert. Have a look at [Becker, Kevin G., et al. "The genetic association database." Nature genetics 36.5 (2004): 431-432.](https://geneticassociationdb.nih.gov/gad.pdf) for details about the database.

## Download data

In [21]:
url = 'https://www.dropbox.com/s/s91q5kp6ausq9cj/REdata.zip?dl=1'

In [23]:
if not os.path.isfile(DATA/'REdata.zip'):
    urllib.request.urlretrieve(url, DATA/'REdata.zip')
    shutil.unpack_archive(DATA/'REdata.zip', extract_dir=DATA)

In [24]:
GAD = DATA/'GAD'

In [29]:
GAD.ls()

(#10) [Path('/home/alex/data-tmp/huggingface/GAD/6'),Path('/home/alex/data-tmp/huggingface/GAD/5'),Path('/home/alex/data-tmp/huggingface/GAD/9'),Path('/home/alex/data-tmp/huggingface/GAD/2'),Path('/home/alex/data-tmp/huggingface/GAD/10'),Path('/home/alex/data-tmp/huggingface/GAD/4'),Path('/home/alex/data-tmp/huggingface/GAD/3'),Path('/home/alex/data-tmp/huggingface/GAD/7'),Path('/home/alex/data-tmp/huggingface/GAD/8'),Path('/home/alex/data-tmp/huggingface/GAD/1')]

## Explore data

In [30]:
pd.set_option('max.colwidth', 1000)
pd.set_option('display.html.use_mathjax', False)

In [31]:
example_fn = GAD/'5'/'train.tsv'
df = pd.read_csv(example_fn, sep='\t', header=None, names=['text', 'interaction'])

In [32]:
df.head()

,text,interaction
0,An interaction with hypertension in the association between the @GENE$ G460W polymorphism and @DISEASE$ merits further testing in additional populations.,1
1,Our study suggests that the five SNPs within @GENE$ gene we studied may not play a major role in the @DISEASE$ susceptibility in the Chinese Han population.,1
2,Our findings suggest that the @GENE$ polymorphism is not associated with an increased risk of squamous cell @DISEASE$ in Korean women.,1
3,Our findings indicate that the @GENE$-112G/A polymorphism does not play a substantial role in genetic predisposition to @DISEASE$ in this Japanese population.,0
4,Although an increasing number of studies report an association between the @GENE$ G1385A variant and @DISEASE$ risk; this variant does not appear to be implicated in the development of breast cancer.,0


In [33]:
df.interaction.value_counts()

1    2521
0    2276
Name: interaction, dtype: int64

## Prepare data

In [34]:
def read_gad_split(df):
    texts = []
    labels = []
    for row in df.iterrows():
        texts.append(row[1].text)
        labels.append(row[1].interaction)
    return texts, labels

In [35]:
train_texts, train_labels = read_gad_split(df)

In [36]:
train_texts[:5]

['An interaction with hypertension in the association between the @GENE$ G460W polymorphism and @DISEASE$ merits further testing in additional populations.',
 'Our study suggests that the five SNPs within @GENE$ gene we studied may not play a major role in the @DISEASE$ susceptibility in the Chinese Han population.',
 'Our findings suggest that the @GENE$ polymorphism is not associated with an increased risk of squamous cell @DISEASE$ in Korean women.',
 'Our findings indicate that the @GENE$-112G/A polymorphism does not play a substantial role in genetic predisposition to @DISEASE$ in this Japanese population.',
 'Although an increasing number of studies report an association between the @GENE$ G1385A variant and @DISEASE$ risk; this variant does not appear to be implicated in the development of breast cancer.']

In [37]:
train_labels[:5]

[1, 1, 1, 0, 0]

In [38]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [39]:
train_encodings = tokenizer(train_texts, padding=True)

In [40]:
val_encodings = tokenizer(val_texts, padding=True)

### Create dataset

In [41]:
import torch

In [42]:
class GADDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
    

In [43]:
train_dataset = GADDataset(train_encodings, train_labels)
val_dataset = GADDataset(val_encodings, val_labels)

## Use the pre-trained transformer model

We'll fine-tune the pre-trained model to classify whether the text indicates an interaction or not. 

In [45]:
MODELS_GDB = DATA/'GAD'/'models'
MODELS_GDB.mkdir(exist_ok=True, parents=True)

In [46]:
from transformers import Trainer, TrainingArguments

In [47]:
training_args = TrainingArguments(
    output_dir=str(MODELS_GDB/'results'),
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=str(MODELS_GDB/'logs'),
    logging_steps=10
    )

In [48]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [49]:
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [50]:
trainer.train()

/home/alex/anaconda3/envs/fastai/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/alex/anaconda3/envs/fastai/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,0.681900
20,0.694100
30,0.692900
40,0.696200
50,0.686600
60,0.659300
70,0.655300
80,0.631700
90,0.612000
100,0.604600


TrainOutput(global_step=120, training_loss=0.643294115861257, metrics={'train_runtime': 51.4329, 'train_samples_per_second': 2.333, 'total_flos': 289861586768340.0, 'epoch': 1.0})

In [51]:
pred = trainer.evaluate(eval_dataset=val_dataset)

In [52]:
pred

{'eval_loss': 0.5877533555030823,
 'eval_accuracy': 0.7114583333333333,
 'eval_f1': 0.7851047323506594,
 'eval_precision': 0.677376171352075,
 'eval_recall': 0.933579335793358,
 'eval_runtime': 2.4287,
 'eval_samples_per_second': 395.272,
 'epoch': 1.0}

# Possible next steps

You can consider trying out the model on other tasks from the benchmark data sets set up by the authors of PubMedBERT [BLURB: Biomedical Language Understanding and Reasoning Benchmark](https://microsoft.github.io/BLURB/) (or any other similar task you may think of).